In [1]:
import pandas as pd
import geopandas as gpd
import folium

In [5]:
import pandas as pd

def dms_to_dd(dms, fallback):
    try:
        parts = dms.split('°')
        degrees = float(parts[0])
        if '′' in parts[1]:
            minutes, seconds = parts[1].split('′')
            if '″' in seconds:
                seconds, _ = seconds.split('″')
            else:
                seconds = 0
        else:
            minutes = parts[1]
            seconds = 0
        dd = degrees + float(minutes)/60 + float(seconds)/(60*60)
        return round(dd, 8)
    except:
        return fallback

# Load the data
df = pd.read_csv('勘災與建管圖資總表_v6.csv')

# drop the rows with x or y is ''
df = df[df['GPS座標E'] != '']
df = df[df['GPS座標N'] != '']

# replace the E and N with empty string
df['GPS座標E'] = df['GPS座標E'].str.replace('E', '')
df['GPS座標N'] = df['GPS座標N'].str.replace('N', '')

# Convert the DMS coordinates to DD
df['GPS_lon'] = df.apply(lambda row: dms_to_dd(row['GPS座標E'], row['Longitude']), axis=1)
df['GPS_lat'] = df.apply(lambda row: dms_to_dd(row['GPS座標N'], row['Latitude']), axis=1)

# print the data if dms_to_dd not return the correct value
if len(df[df['GPS_lon'] == 'Longitude']) > 0 or len(df[df['GPS_lat'] == 'Latitude']) > 0:
    print(df[df['GPS_lon'] == 'Longitude'])
    print(df[df['GPS_lat'] == 'Latitude']) 
else:
    print('All coordinates are converted successfully')

# save the data
df.to_csv('勘災與建管圖資總表_v6_edit.csv', index=False)


All coordinates are converted successfully


In [7]:
# Read the CSV files
buildings = gpd.read_file('/Users/user1/Desktop/yang/0403_data/map/勘災與建管圖資總表_v6_edit.csv')
center = gpd.read_file('/Users/user1/Desktop/yang/0403_data/map/震央.csv')
stations = gpd.read_file('/Users/user1/Desktop/yang/0403_data/map/測站查詢_638482961965651103.csv')

# print columns of the data
print(buildings.columns)

# Convert 'class' to integer
buildings['class'] = buildings['class'].astype(int)

# drop the rows with GPS_lat or GPS_lon is ''
buildings = buildings[buildings['GPS_lat'] != '']
buildings = buildings[buildings['GPS_lon'] != '']

# only keep the col '測站代碼' is HWA in stations
stations = stations[stations['測站代碼'] == 'HWA']

# set the col 'Latitude' and 'Longitude' to float
buildings['GPS_lat'] = buildings['GPS_lat'].astype(float)
buildings['GPS_lon'] = buildings['GPS_lon'].astype(float)
center['lat'] = center['lat'].astype(float)
center['lon'] = center['lon'].astype(float)
stations['緯度'] = stations['緯度'].astype(float)
stations['經度'] = stations['經度'].astype(float)

# Create a map centered around the first building
m = folium.Map(location=[buildings['GPS_lat'].mean(), buildings['GPS_lon'].mean()], zoom_start=13)

# Add the buildings to the map
for _, building in buildings.iterrows():
    if building['class'] == 1:
        shape = folium.RegularPolygonMarker(
            location=[building['GPS_lat'], building['GPS_lon']],
            number_of_sides=4,
            radius=5,
            color='green',
            fill_opacity=1,
            fill=False,
            weight=2
        )
    elif building['class'] == 2:
        shape = folium.RegularPolygonMarker(
            location=[building['GPS_lat'], building['GPS_lon']],
            number_of_sides=3,
            radius=5,
            color='black',
            fill=True,
            fill_opacity=1,
            fill_color='yellow',
            weight=1
        )
    else:
        shape = folium.CircleMarker(
            location=[building['GPS_lat'], building['GPS_lon']],
            radius=5,
            color='black',
            fill=True,
            fill_color='red',
            fill_opacity=1,
            weight=1
        )
    shape.add_to(m)
    folium.Popup(f"建物名稱: {building['建物名稱']}<br>地址: {building['地址']}<br>樓層: {building['樓層']}", max_width=250).add_to(shape)

# Add the center point to the map
for _, c in center.iterrows():
    marker = folium.Marker(
        location=[c['lat'], c['lon']],
        icon=folium.Icon(color='red', icon='star')
    )
    marker.add_to(m)
    folium.Popup(f"發震時間：2024/04/03 07:58:09<br>芮氏規模：7.2", max_width=250).add_to(marker)

# Add the stations to the map
for _, station in stations.iterrows():
    shape = folium.CircleMarker(
        location=[station['緯度'], station['經度']],
        radius=5,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=1
    )
    shape.add_to(m)
    folium.Popup(f"測站代碼: {station['測站代碼']}<br>測站名稱: {station['測站名稱']}<br>位置: {station['詳細地址']}", max_width=250).add_to(shape)

# Create a legend
legend_html = """
<div style="position: fixed; bottom: 50px; right: 50px; width: 200px; height: 150px; border:2px solid grey; z-index:9999; font-size:14px;">
<p><a style="color:green; text-decoration:none;">&#9724;</a>&nbsp; 綠單（未貼單/已補強）</p>
<p><a style="color:yellow; text-decoration:none;">&#9650;</a>&nbsp; 黃單</p>
<p><a style="color:red; text-decoration:none;">&#9679;</a>&nbsp; 紅單</p>
<p><a style="color:red; text-decoration:none;">&#9733;</a>&nbsp; 震央</p>
<p><a style="color:blue; text-decoration:none;">&#9679;</a>&nbsp; 花蓮觀測站(HWA)</p>
</div>
"""
legend = folium.Element(legend_html)
m.get_root().html.add_child(legend)  

# Save the map to an HTML file
m.save('map.html')

Index(['編號', '使用執照號碼', '原建照編號', '棟別編號', '勘災組編號', '建物名稱', '地址', 'Latitude',
       'Longitude', '建築物用途說明', '高度', '樓層', '分數', '風險等級', '張貼紅色危險標誌',
       '張貼黃色危險標誌', '綠標(未貼單/已補強)', 'class', '預定勘災日', '紅黃單有無使照註記', 'GPS座標E',
       'GPS座標N', '勘災人員現場震損判斷', '上傳NCREE', 'drawing', 'photo', 'GPS_lon',
       'GPS_lat', 'geometry'],
      dtype='object')
